In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection

from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import  mean_squared_error  
from sklearn import datasets

In [ ]:
train_data=pd.read_csv('/content/drive/MyDrive/Wipro hackathon/Participants_Data_WSMLC/train.csv')
test_data=pd.read_csv('/content/drive/MyDrive/Wipro hackathon/Participants_Data_WSMLC/test.csv')

In [ ]:
features=train_data[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity','Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]
features.head()

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009,1,1,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0
1,2009,1,1,0,30,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0
2,2009,1,1,1,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0
3,2009,1,1,1,30,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0
4,2009,1,1,2,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0


In [ ]:
train_y=train_data[['Clearsky DHI','Clearsky DNI','Clearsky GHI']]
train_y.describe()

,Clearsky DHI,Clearsky DNI,Clearsky GHI
count,175296.000000,175296.000000,175296.000000
mean,53.065426,330.065295,254.365490
std,64.201407,370.030821,327.705006
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,103.000000,736.000000,535.000000
max,565.000000,1040.000000,1063.000000


In [ ]:
def create_folds(data, num_splits):
    data["kfold"] = -1
    num_bins = int(np.floor(1 + np.log2(len(data))))

    data.loc[:, "bins"] = pd.cut(data["Clearsky DHI"], bins=num_bins, labels=False)

    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    data = data.drop("bins", axis=1)

    return data

In [ ]:
train_data_dhi=create_folds(train_data,5)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


In [ ]:
model_dhi1=XGBRegressor(n_estimators=1200,random_state=42)
model_dhi2=XGBRegressor(n_estimators=1200,random_state=42)
model_dhi3=XGBRegressor(n_estimators=1200,random_state=42)
model_dhi4=XGBRegressor(n_estimators=1200,random_state=42)
model_dhi5=XGBRegressor(n_estimators=1200,random_state=42)
errors_dhi=[]
models=[]
for i in range(5):
  train_data=train_data_dhi[train_data_dhi['kfold']!=i]
  train_data_y=train_data['Clearsky DHI']
  train_data_x=train_data[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity','Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]
  valid_data=train_data_dhi[train_data_dhi['kfold']==i]
  valid_data_y=valid_data['Clearsky DHI']
  valid_data_x=valid_data[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity','Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]
  print(train_data.shape,valid_data.shape)
  #model_dhi=XGBRegressor(n_estimators=1200,random_state=42)
  if(i==0):
    model_dhi1.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dhi1.predict(valid_data_x)
    errors_dhi.append(mean_squared_error(predict,valid_data_y))

  if(i==1):

    model_dhi2.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dhi2.predict(valid_data_x)
    errors_dhi.append(mean_squared_error(predict,valid_data_y))

  if(i==2):
    model_dhi3.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dhi3.predict(valid_data_x)
    errors_dhi.append(mean_squared_error(predict,valid_data_y))

  if(i==3):
    model_dhi4.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dhi4.predict(valid_data_x)
    errors_dhi.append(mean_squared_error(predict,valid_data_y))

  if(i==4):
    model_dhi5.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dhi5.predict(valid_data_x)
    errors_dhi.append(mean_squared_error(predict,valid_data_y))
   

(140236, 19) (35060, 19)
[16:23:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:24:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:25:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:26:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:27:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
errors_dhi

[195.28015370926246,
 200.33780737316243,
 190.69357459019704,
 191.74020444668258,
 197.21837000489415]

In [ ]:
train_data=pd.read_csv('/content/drive/MyDrive/Wipro hackathon/Participants_Data_WSMLC/train.csv')
test_data=pd.read_csv('/content/drive/MyDrive/Wipro hackathon/Participants_Data_WSMLC/test.csv')

In [ ]:
def create_folds2(data, num_splits):
    data["kfold"] = -1
    num_bins = int(np.floor(1 + np.log2(len(data))))

    data.loc[:, "bins"] = pd.cut(data["Clearsky DNI"], bins=num_bins, labels=False)

    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    data = data.drop("bins", axis=1)

    return data

In [ ]:
train_data_dni=create_folds2(train_data,5)

In [ ]:
model_dni1=XGBRegressor(n_estimators=1200,random_state=42)
model_dni2=XGBRegressor(n_estimators=1200,random_state=42)
model_dni3=XGBRegressor(n_estimators=1200,random_state=42)
model_dni4=XGBRegressor(n_estimators=1200,random_state=42)
model_dni5=XGBRegressor(n_estimators=1200,random_state=42)
errors_dni=[]
models=[]
for i in range(5):
  train_data=train_data_dni[train_data_dni['kfold']!=i]
  train_data_y=train_data['Clearsky DNI']
  train_data_x=train_data[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity','Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]
  valid_data=train_data_dni[train_data_dni['kfold']==i]
  valid_data_y=valid_data['Clearsky DNI']
  valid_data_x=valid_data[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity','Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]
  print(train_data.shape,valid_data.shape)
  #model_dhi=XGBRegressor(n_estimators=1200,random_state=42)
  if(i==0):
    model_dni1.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dni1.predict(valid_data_x)
    errors_dni.append(mean_squared_error(predict,valid_data_y))

  if(i==1):

    model_dni2.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dni2.predict(valid_data_x)
    errors_dni.append(mean_squared_error(predict,valid_data_y))

  if(i==2):
    model_dni3.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dni3.predict(valid_data_x)
    errors_dni.append(mean_squared_error(predict,valid_data_y))

  if(i==3):
    model_dni4.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dni4.predict(valid_data_x)
    errors_dni.append(mean_squared_error(predict,valid_data_y))

  if(i==4):
    model_dni5.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_dni5.predict(valid_data_x)
    errors_dni.append(mean_squared_error(predict,valid_data_y))
   

(140236, 19) (35060, 19)
[16:41:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:43:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:44:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:46:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:48:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
errors_dni

[1143.8714882866357,
 1190.271859767755,
 1116.3508734704237,
 1191.3636527948395,
 1154.7643258881149]

In [ ]:
train_data=pd.read_csv('/content/drive/MyDrive/Wipro hackathon/Participants_Data_WSMLC/train.csv')
test_data=pd.read_csv('/content/drive/MyDrive/Wipro hackathon/Participants_Data_WSMLC/test.csv')

In [ ]:
def create_folds3(data, num_splits):
    data["kfold"] = -1
    num_bins = int(np.floor(1 + np.log2(len(data))))

    data.loc[:, "bins"] = pd.cut(data["Clearsky GHI"], bins=num_bins, labels=False)

    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    data = data.drop("bins", axis=1)

    return data

In [ ]:
train_data_ghi=create_folds3(train_data,5)

In [ ]:
model_ghi1=XGBRegressor(n_estimators=1200,random_state=42)
model_ghi2=XGBRegressor(n_estimators=1200,random_state=42)
model_ghi3=XGBRegressor(n_estimators=1200,random_state=42)
model_ghi4=XGBRegressor(n_estimators=1200,random_state=42)
model_ghi5=XGBRegressor(n_estimators=1200,random_state=42)
errors_ghi=[]
 
for i in range(5):
  train_data=train_data_ghi[train_data_ghi['kfold']!=i]
  train_data_y=train_data['Clearsky GHI']
  train_data_x=train_data[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity','Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]
  valid_data=train_data_ghi[train_data_ghi['kfold']==i]
  valid_data_y=valid_data['Clearsky GHI']
  valid_data_x=valid_data[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity','Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]
  print(train_data.shape,valid_data.shape)
  #model_dhi=XGBRegressor(n_estimators=1200,random_state=42)
  if(i==0):
    model_ghi1.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_ghi1.predict(valid_data_x)
    errors_ghi.append(mean_squared_error(predict,valid_data_y))

  if(i==1):

    model_ghi2.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_ghi2.predict(valid_data_x)
    errors_ghi.append(mean_squared_error(predict,valid_data_y))

  if(i==2):
    model_ghi3.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_ghi3.predict(valid_data_x)
    errors_ghi.append(mean_squared_error(predict,valid_data_y))

  if(i==3):
    model_ghi4.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_ghi4.predict(valid_data_x)
    errors_ghi.append(mean_squared_error(predict,valid_data_y))

  if(i==4):
    model_ghi5.fit(train_data_x,train_data_y,early_stopping_rounds=4,eval_set=[(valid_data_x,valid_data_y)],verbose=False)
    predict=model_ghi5.predict(valid_data_x)
    errors_ghi.append(mean_squared_error(predict,valid_data_y))
   

(140236, 19) (35060, 19)
[16:54:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:56:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:57:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[16:59:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(140237, 19) (35059, 19)
[17:01:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
errors_ghi

[32.43706485624388,
 32.829726441908925,
 32.41942991391489,
 32.11845958523422,
 32.74228065401354]

In [ ]:
test_data_x=test_data[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity','Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]

In [ ]:
predict_dhi1=model_dhi1.predict(test_data_x)
predict_dhi2=model_dhi2.predict(test_data_x)
predict_dhi3=model_dhi3.predict(test_data_x)
predict_dhi4=model_dhi4.predict(test_data_x)
predict_dhi5=model_dhi5.predict(test_data_x)
predict_dhi=(predict_dhi1+predict_dhi2+predict_dhi3+predict_dhi4+predict_dhi5)/5.0 

In [ ]:
predict_dni1=model_dni1.predict(test_data_x)
predict_dni2=model_dni2.predict(test_data_x)
predict_dni3=model_dni3.predict(test_data_x)
predict_dni4=model_dni4.predict(test_data_x)
predict_dni5=model_dni5.predict(test_data_x)
predict_dni=(predict_dni1+predict_dni2+predict_dni3+predict_dni4+predict_dni5)/5.0 

In [ ]:
predict_ghi1=model_ghi1.predict(test_data_x)
predict_ghi2=model_ghi2.predict(test_data_x)
predict_ghi3=model_ghi3.predict(test_data_x)
predict_ghi4=model_ghi4.predict(test_data_x)
predict_ghi5=model_ghi5.predict(test_data_x)
predict_ghi=(predict_ghi1+predict_ghi2+predict_ghi3+predict_ghi4+predict_ghi5)/5.0 

In [ ]:
for i in range(predict_dhi.shape[0]):
  if(predict_dhi[i]<0):
    predict_dhi[i]=0

for i in range(predict_dni.shape[0]):
  if(predict_dni[i]<0):
    predict_dni[i]=0

for i in range(predict_ghi.shape[0]):
  if(predict_ghi[i]<0):
    predict_ghi[i]=0


In [ ]:
predict={'Clearsky DHI':predict_dhi,'Clearsky DNI':predict_dni,'Clearsky GHI':predict_ghi}
submit=pd.DataFrame(predict)

In [ ]:
submit.describe()

,Clearsky DHI,Clearsky DNI,Clearsky GHI
count,17520.000000,17520.000000,17520.000000
mean,49.752232,341.809784,256.332184
std,56.982178,374.809753,328.937073
min,0.000000,0.000000,0.000000
25%,0.000000,1.045866,0.220013
50%,7.077966,28.838941,4.522556
75%,99.412123,760.534607,536.745758
max,248.491974,1022.593384,1036.334473


In [ ]:
submit.to_csv('submission7.csv',index=False)